In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
batch_size = 64

<h3>data</h3>

In [3]:
train_dataset = datasets.MNIST(root='./mnist_data',
                              train = True,
                              transform = transforms.ToTensor(),
                              download=True)

In [4]:
test_dataset = datasets.MNIST(root='./mnist_data',
                             train=False,
                             transform=transforms.ToTensor())

In [5]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)

In [6]:
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                       batch_size=batch_size,
                                       shuffle=False)

<h3>model</h3>

In [7]:
class InceptionA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.branch5x5_2 = nn.Conv2d(16,24,kernel_size=5, padding=2)
        
        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16,24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24,24, kernel_size=3, padding=1)
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
    
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
        
        branch_pool = F.avg_pool2d(x,kernel_size=3,stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs,1)

In [8]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [9]:
model = Net()

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0]))

In [12]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
for epoch in range(1, 10):
    train(epoch)
    test()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304836
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.306655
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.312030
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.317965
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.296031
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.303312
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.304923
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.302845
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.294347
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.284154
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.282834
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.288178
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.289871
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.262360
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.254781
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.243485
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.244466
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.179948
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.161572
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.133821
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.087559
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.110316
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.113113
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.038099
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.136679
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.070465
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.240618
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.260805
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.167065
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.080424
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.083216
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.130950
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.207873
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.068241
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.180589
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.087231
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.048346
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.052525
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.066541
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.200490


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.071286
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.101480
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.117464
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.024853
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.185186
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.039711
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.022175
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.074675
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.065084
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.079346
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.206564
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.038287
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.179718
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.025913
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.157076
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.041337
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.030898
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.071631
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.128145
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.096665


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.184702
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.048839
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.011561
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.097026
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.034877
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.091270
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.043541
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.034134
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.103660
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.062894
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.106891
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.040939
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.062741
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.032596
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.031298
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.086295
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.044079
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.010467
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.075942
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.050558


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.034074
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.228142
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.131900
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.153565

Test set: Average loss: 0.0538, Accuracy: 9833/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.126220
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.045364
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.014144
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.153917
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.041113
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.018071
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.093675
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.008579
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.005953
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.036559
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.050363
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.166980
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.050735
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.101722
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.090256
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.048019
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.111901
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.009450
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.036548
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.036588
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.015022
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.022731
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.144102
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.046714
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.017033
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003019
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.019810
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.142899
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.060050
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.110628
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.074172
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.005426
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.019855
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.030711
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.173216
